In [20]:
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import pandas as pd
df = pd.read_csv('../input/titanic/train.csv')

In [21]:
df.head()
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [22]:
print(df.head())

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [23]:
by_sex = df.groupby('Sex')
for name, group_data in by_sex:
    print('Nhóm: ',name)
    print(group_data.head(2))
    print('/n')


Nhóm:  female
   PassengerId  Survived  Pclass  \
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
/n
Nhóm:  male
   PassengerId  Survived  Pclass                      Name   Sex   Age  SibSp  \
0            1         0       3   Braund, Mr. Owen Harris  male  22.0      1   
4            5         0       3  Allen, Mr. William Henry  male  35.0      0   

   Parch     Ticket  Fare Cabin Embarked  
0      0  A/5 21171  7.25   NaN        S  
4      0     373450  8.05   NaN        S  
/n


In [24]:
survival_rate_sex = by_sex['Survived'].mean()
print(survival_rate_sex)    

Sex
female    0.742038
male      0.188908
Name: Survived, dtype: float64


In [25]:
import numpy as np 
df['AgeGroup'] = np.where(df['Age'] < 18, 'Child', 'Adult')
print(df[['Age', 'AgeGroup', 'Sex']].head())

    Age AgeGroup     Sex
0  22.0    Adult    male
1  38.0    Adult  female
2  26.0    Adult  female
3  35.0    Adult  female
4  35.0    Adult    male


/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in less
  return op(a, b)


In [26]:
survival_rate_age = df.groupby('AgeGroup')['Survived'].mean()
print(survival_rate_age)

AgeGroup
Adult    0.361183
Child    0.539823
Name: Survived, dtype: float64


In [27]:
survival_rate_ageandsex = df.groupby(['AgeGroup','Sex'])['Survived'].mean()
print(survival_rate_ageandsex)

AgeGroup  Sex   
Adult     female    0.752896
          male      0.165703
Child     female    0.690909
          male      0.396552
Name: Survived, dtype: float64


In [28]:
df_test = pd.read_csv('../input/titanic/test.csv')
df_test['Survived'] = np.where(df_test['Sex'] == 'female',1 ,0)
print(df_test[['PassengerId', 'Sex', 'Survived']].head())

   PassengerId     Sex  Survived
0          892    male         0
1          893  female         1
2          894    male         0
3          895    male         0
4          896  female         1


In [29]:
submission = df_test[['PassengerId','Survived']]
submission.to_csv('submission.csv',index = False)

In [30]:
df['Title'] = df['Name'].apply(lambda name : name.split(',')[1].split('.')[0].strip())
df_test['Title'] = df_test['Name'].apply(lambda name : name.split(',')[1].split('.')[0].strip())
print(df['Title'].value_counts())

Title
Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Mlle              2
Major             2
Col               2
the Countess      1
Capt              1
Ms                1
Sir               1
Lady              1
Mme               1
Don               1
Jonkheer          1
Name: count, dtype: int64


In [31]:
for data in [df,df_test]:
    data['Title'] = data['Title'].replace(['Mlle','Ms'],'Miss')
    data['Title'] = data['Title'].replace('Mme','Mrs')
    rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Lady', 'the Countess', 
                   'Jonkheer', 'Don', 'Capt', 'Sir']
    data['Title'] = data['Title'].replace(rare_titles, 'Rare')
print(df['Title'].value_counts())

Title
Mr        517
Miss      185
Mrs       126
Master     40
Rare       23
Name: count, dtype: int64


In [32]:
title_dummies_train = pd.get_dummies(df['Title'], prefix = 'Title')
title_dummies_test = pd.get_dummies(df_test['Title'], prefix = 'Title')
df = pd.concat([df,title_dummies_train],axis = 1)
df_test = pd.concat([df_test,title_dummies_train],axis = 1)
df.drop('Title',axis = 1, inplace = True)
df_test.drop('Title',axis = 1, inplace = True)
print(list(df.columns[-5:]))

['Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare']


In [33]:
feature = ['Pclass','Sex','Age','SibSp','Parch','Fare','Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs', 'Title_Rare']
df['Sex'] = df['Sex'].map({'male':0,'female':1})
df_test['Sex'] = df_test['Sex'].map({'male':0,'female':1})

In [34]:
print(df[feature].isnull().sum())
print(df_test[feature].isnull().sum())

Pclass            0
Sex               0
Age             177
SibSp             0
Parch             0
Fare              0
Title_Master      0
Title_Miss        0
Title_Mr          0
Title_Mrs         0
Title_Rare        0
dtype: int64
Pclass          473
Sex             473
Age             559
SibSp           473
Parch           473
Fare            474
Title_Master      0
Title_Miss        0
Title_Mr          0
Title_Mrs         0
Title_Rare        0
dtype: int64


In [35]:
median_age = df['Age'].median()
median_fare = df['Fare'].median()

In [36]:
df['Age'] = df['Age'].fillna(median_age)
df_test['Age'] = df_test['Age'].fillna(median_age)
df_test['Fare'] = df_test['Fare'].fillna(median_fare)

In [37]:
X_train = df[feature]
Y_train = df['Survived']
model = XGBClassifier(n_estimators = 100,learning_rate = 0.1,random_state = 1)
model.fit(X_train, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=1, ...)

In [38]:
X_test = df_test[feature]
Y_test = df_test['Survived']
prediction = model.predict(X_test)

In [39]:
submission = pd.DataFrame({"PassengerId" : df_test["PassengerId"],
"Survived" : prediction})
submission.to_csv('submission_xgboost_title.csv', index = False)